# Using  EAGO with a script-defined problem:  
### *Implicit relaxation of state functions*

[Matthew Wilhelm](https://psor.uconn.edu/person/matthew-wilhelm/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

EAGO includes a specialized series of routines which can solve certain optimization problems by relaxing implicit functions directly without the introduction of additional auxilliary variables. The implicit functions, $\mathbf{x(p)}$ are defined by a series of equalities $\mathbf{h}(\mathbf{x},\mathbf{p}) = \mathbf{0}$ on some domain $(X \times P)$ allow for the problem to be treated implicitly using the below form. While exact evaluations of the objective function and constraints in this form are difficult, relaxations used in global optimization are much easier to calculate. In some cases, this problem may be much easier to solve when formulated in this manner due to a significant reduction in decision variable space (in cases where $n_x >> n_p$). 

$
\begin{align}\label{form:2}
    f^*=&\min_{\mathbf p \in P}f(\mathbf x(\mathbf p),\mathbf p)\\
    &{\rm s.t.}\; \mathbf{g(x(p),p)\le 0}.
\end{align}
$

## Formulate objective
We formulate the objective as a regular function of the state variables $\mathbf{x}$ and the control variables $\mathbf{p}$.

In [ ]:
using EAGO, MathOptInterface

f(x, p) = x[1]

## Define the implicit function & jacobian of state function

We now define the function $\mathbf{h}(\mathbf{x},\mathbf{p}) = \mathbf{0}$ which defines the state variable. EAGO's implicit solver assumes this function is defined in place as a function of the output variables, state variables, and control variables, respectively.

In [ ]:
function h!(out, x, p)
    out[1] = x[1]^2 + x[1]*p[1] + 4.0
end

We now define the the jacobian of the state function *w.r.t* to state variables only. This function must be supplied in place as this allows better integration with different linear algebra libraries. **EAGO** will use an automated differentiation library to calculate this function if the user does not provide it. However, some performance improvement is expected when a user-defined function is supplied. Typically, this improvement is on the order of 3-10x.

In [ ]:
function hj!(out, x, p)
    out[1] = 2.0*x[1] + p[1]
end

## Solve the implicit formulation

In [ ]:
p_L = [6.0];
p_U = [9.0];
x_L = [-0.78];
x_U = [-0.4]

opt = EAGO.Optimizer(absolute_tolerance=1e-3)
var, opt = solve_implicit(f, h!, x_L, x_U, p_L, p_U, opt, hj!, nothing)

## Retrieve solution information

As this solver interfaces directly with the MOI EAGO Optimizer to support a nonstandard form, information from the solution is obtained by using the MOI access functions.

In [ ]:
# get the primal status code
primal_status = MOI.get(opt, MOI.PrimalStatus())

# get the termination status code
termination_status = MOI.get(opt, MOI.TerminationStatus())

# retrieve the objective value
obj_value = MOI.get(opt, MOI.ObjectiveValue())

# retrieve the primal variable results for the decision variables (nx+1):(nx+np)
primal_variable_result = MOI.get(opt, MOI.VariablePrimal(), var[(1:nx):(nx+np)])

println("The primal status is $(primal_status)")
println("The termination status is $(termination_status)")
println("The objective value is $(obj_value)")
println("The primal variable result is $(primal_variable_result)")

### Reference:
1. 1. J. W. Taylor, et al., Direct measurement of the fast, reversible addition of oxygen to cyclohexadienyl radicals in nonpolar solvents, *J. Phys. Chem. A.*, **2004**, 108, 7193–7203.
2. A. B. Singer, et al., Global dynamic optimization for parameter estimation in chemical kinetics, *J. Phys. Chem A.*, **2006**, 110, 971–976.
3. Mitsos, A. Chachuat, B., & Barton, P.I., McCormick-based relaxations of algorithms, *SIAM Journal on Optimization*, *SIAM*, **2009**, 20(2), 573-601.
4. Stuber, M.D., Scott, J.K., & Barton, P.I.: Convex and concave relaxations of implicit functions. *Optim. Methods Softw.*, **2015** 30(3), 424–460.